The following cell loads in all the libraries that we previously installed; this makes it so that we can access the functions in these libraries instead of having to code these capabilities from scratch.

In [9]:
import csv
import cv2
import numpy as np
import librosa
import librosa.display
import wave
import os
import matplotlib.pyplot as plt
import multiprocessing 
import pickle

The next cell defines the folder paths to both the source folder where the audio is stored and the folder where the generated images should go.

In [10]:
SOURCE_FOLDER_TRAIN= '../AudioData/Cats-Vs-Dogs/'
OUTPUT_FOLDER_TRAIN = '../GeneratedData/Cats-Vs-Dogs/'

The following cell defines the filename where the Spectrum Variables should be read from. This will load in a file that tells this script how to compute the spectrogram.

In [11]:
#Loading in the Spectrogram variables
FileName =input("Please type filename without the file ending here => ")
SpectrumVariables = pickle.load(open( FileName+'.SpecVar', "rb" ) )
SpectrumVariables

Please type filename without the file ending here => Standard


{'RESOLUTION': 224,
 'SAMPLE_RATE': 48000,
 'N_FFT': 3000,
 'HOP_LENGTH': 512,
 'FMIN': 0,
 'FMAX': 24000,
 'N_MELS': 224,
 'POWER': 2.1}

This defines the function that creates logarithmic spectragrams of the audio file designated in the `src_path` into the folder in the `dst_path`.

In [12]:
def log_mel_spec_tfm(src_path, dst_path):
    #print(src_path)
    fname=src_path.split('/')[-1];
    x=[]
    sample_rate=0
    try:
        channels ,sample_rate_in = librosa.load(src_path,mono=False)
    except:
        print(fname+" Could not be computed!")
        return
    if(channels.ndim==1):
        channels= channels.reshape((1,(len(channels))))
    channelcounter=0
    for audio_data in channels:
        channelcounter+=1
        
        RESOLUTION=SpectrumVariables["RESOLUTION"] 
        
        N_FFT=SpectrumVariables["N_FFT"]
        HOP_LENGTH= SpectrumVariables["HOP_LENGTH"]
        FMIN=SpectrumVariables["FMIN"]
        FMAX=SpectrumVariables["FMAX"]
        N_MELS=SpectrumVariables["N_MELS"]
        POWER=SpectrumVariables["POWER"]
        #audio_data = librosa.resample(x, sample_rate_in, SAMPLE_RATE)
        mel_spec_power = librosa.feature.melspectrogram(np.array(audio_data), sr=sample_rate_in, 
                                                        n_mels=N_MELS)
        mel_spec_db = librosa.power_to_db(mel_spec_power, ref=np.max)
        image = mel_spec_db; # convert to float
        image -= image.min() # ensure the minimal value is 0.0
        image /= image.max() # maximum value in image is now 1.0
        image*=256;
        img = image.astype(np.uint8)
        colorPic = cv2.applyColorMap(img, cv2.COLORMAP_BONE)
        #cv2.imshow('dst_rt', colorPic)
        #cv2.waitKey(0)
        count=0
        for i in range(int(np.floor(colorPic.shape[1]/RESOLUTION))):
            count+=1
            startx=RESOLUTION*i
            stopx=RESOLUTION*(i+1)
            OutputImage = cv2.resize(colorPic[:,startx:stopx,:],(RESOLUTION,RESOLUTION))
            plt.imsave(os.path.join(dst_path,(fname.replace(".wav",'-').replace(".m4a",'-').replace(".mp3",'-') +str(i)+'-ch-'+str(channelcounter)+ '.png')), OutputImage) 
        if(count==0):
            print(src_path)
            #print("WARNING: => Jumped A file. If you see this often, increas sampleRate or Hop length.")

The following folder is a wrapper function for the previous function, going through all the audio files in `IN_FOLDER`.

In [7]:
def ComputeSpectrograms(IN_FOLDER,OUT_FOLDER):
    print("I will print the file path to files that are too short to fit into one full picture.")
    SourceFoldersLabels = [f.path for f in os.scandir(IN_FOLDER) if f.is_dir()]
    for path in SourceFoldersLabels:
        FileList = [f.path for f in os.scandir(path) if f.is_file() and (f.name.endswith(".wav") or f.name.endswith(".m4a") or f.name.endswith(".mp3"))]
        Label = path.split('/')[-1]
        outFolder = os.path.join(OUT_FOLDER,Label)
        print(FileList)
        if not os.path.exists(outFolder):
            os.makedirs(outFolder)
        #with multiprocessing.Pool(12) as p:
            #p.starmap(log_mel_spec_tfm, [(f,outFolder) for f in FileList])
        for f in FileList:
            log_mel_spec_tfm(f,outFolder)
        print("Finished this class. Going to the next")
    print("Done generating spectra!")  

This moves the Spectrum Variables file into the output folder so that we can correspond the images with labels.

In [8]:
ComputeSpectrograms(SOURCE_FOLDER_TRAIN,OUTPUT_FOLDER_TRAIN)
pickle.dump(SpectrumVariables, open(os.path.join(OUTPUT_FOLDER_TRAIN,'Main.SpecVar'), "wb" ))
print("Notebook Done")

I will print the file path to files that are too short to fit into one full picture.
['../AudioData/Cats-Vs-Dogs/Dogs/446370__georra43__dogs-barking.wav', '../AudioData/Cats-Vs-Dogs/Dogs/122104__rickyjezz__dog-barking-at-night-nice-reflection.wav', '../AudioData/Cats-Vs-Dogs/Dogs/347763__apolloaiello__dog-barking.wav', '../AudioData/Cats-Vs-Dogs/Dogs/166421__felix-blume__some-dogs-barking-a-village-in-anatolia-turkey.wav', '../AudioData/Cats-Vs-Dogs/Dogs/327666__juan-merie-venter__dog-bark.wav', '../AudioData/Cats-Vs-Dogs/Dogs/415022__roman-cgr__barking-dog.wav', '../AudioData/Cats-Vs-Dogs/Dogs/163459__littlebigsounds__lbs-fx-dog-small-alert-bark001.wav', '../AudioData/Cats-Vs-Dogs/Dogs/231762__davidmenke__fx-dog-barking.wav', '../AudioData/Cats-Vs-Dogs/Dogs/276267__apolloaiello__mr-dog-01.wav', '../AudioData/Cats-Vs-Dogs/Dogs/175917__conleec__amb-s-dog-park-busy-001.wav', '../AudioData/Cats-Vs-Dogs/Dogs/175916__conleec__amb-s-dog-park-busy-002.wav', '../AudioData/Cats-Vs-Dogs/Dogs/175

/Users/dg/venvs/py37/lib/python3.7/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
/Users/dg/venvs/py37/lib/python3.7/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')


../AudioData/Cats-Vs-Dogs/Cats/16466__acclivity__theladysaysno.wav


/Users/dg/venvs/py37/lib/python3.7/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
/Users/dg/venvs/py37/lib/python3.7/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')


../AudioData/Cats-Vs-Dogs/Cats/461545__slv1__cat-meowing.wav
../AudioData/Cats-Vs-Dogs/Cats/461545__slv1__cat-meowing.wav
../AudioData/Cats-Vs-Dogs/Cats/362953__josepharaoh99__cat-call-meow.mp3
../AudioData/Cats-Vs-Dogs/Cats/362953__josepharaoh99__cat-call-meow.mp3
../AudioData/Cats-Vs-Dogs/Cats/163286__dancip__meow-sample.wav
../AudioData/Cats-Vs-Dogs/Cats/163286__dancip__meow-sample.wav


...

/Users/dg/venvs/py37/lib/python3.7/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')


../AudioData/Cats-Vs-Dogs/Cats/240293__xtrgamr__catmeow-01.wav
../AudioData/Cats-Vs-Dogs/Cats/448018__lupsi__female-cat-short-meoow.mp3


/Users/dg/venvs/py37/lib/python3.7/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')


../AudioData/Cats-Vs-Dogs/Cats/55150__noisecollector__chino2.wav
../AudioData/Cats-Vs-Dogs/Cats/407188__spyboticer__cat-prr-and-meow.wav
../AudioData/Cats-Vs-Dogs/Cats/407188__spyboticer__cat-prr-and-meow.wav
../AudioData/Cats-Vs-Dogs/Cats/260179__swatkamus__kitty-meow.wav
../AudioData/Cats-Vs-Dogs/Cats/260179__swatkamus__kitty-meow.wav
../AudioData/Cats-Vs-Dogs/Cats/368013__trngle__meow-5.wav
../AudioData/Cats-Vs-Dogs/Cats/368013__trngle__meow-5.wav
../AudioData/Cats-Vs-Dogs/Cats/61259__timtube__cat-meowing.wav
../AudioData/Cats-Vs-Dogs/Cats/338272__andreas-mustola__mjau4.wav
../AudioData/Cats-Vs-Dogs/Cats/338272__andreas-mustola__mjau4.wav
../AudioData/Cats-Vs-Dogs/Cats/222749__jenpisco__video-3.mp3
../AudioData/Cats-Vs-Dogs/Cats/222749__jenpisco__video-3.mp3
Finished this class. Going to the next
Done generating spectra!
Notebook Done


/Users/dg/venvs/py37/lib/python3.7/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')


In [13]:
pickle.dump(SpectrumVariables, open(os.path.join(OUTPUT_FOLDER_TRAIN,'Main.SpecVar'), "wb" ))

In [ ]:
print('test')